In [27]:
import torch
from collections import OrderedDict

import numpy as np
import warnings
import time
from torch.autograd import grad
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from torch.utils.data import DataLoader, TensorDataset

In [28]:
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()

        # parameters
        self.depth = len(layers) - 1

        # set up layer order dict
        self.activation = torch.nn.Tanh

        layer_list = list()
        for i in range(self.depth - 1):
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))

        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)

        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)

    def forward(self, x):
        out = self.layers(x)
        return out

class PhysicsInformedNN():
    def __init__(self, X, Xb, ub, layers, lr, device):

        self.device = device

        # data trong miền
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.y = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        # data trên biên
        self.xb = torch.tensor(Xb[:, 0:1], requires_grad=True).float().to(device)
        self.yb = torch.tensor(Xb[:, 1:2], requires_grad=True).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)



        # deep neural networks
        self.dnn = DNN(layers).to(device)
        self.lr = lr

        # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(),
            lr=1.0,
            max_iter=20,
            history_size=50,
            tolerance_grad=1e-5,
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )

        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters())
        self.iter = 0
        self.losses_hist = []

    def net_u(self, x, y):

        outputs = self.dnn(torch.cat([x,y], dim=1))
        u = outputs[:, 0]
        return u

    def net_f(self, x, y, xb, yb):
        u = self.net_u(x, y)
        u_b = self.net_u(xb, yb)

        u_x, u_y = torch.autograd.grad(outputs=u, inputs=[x, y],
                                            grad_outputs=torch.ones_like(u),
                                            retain_graph=True, create_graph=True)

        u_xx = torch.autograd.grad(outputs=u_x, inputs=x, grad_outputs=torch.ones_like(u_x),
                                   retain_graph=True, create_graph=True)[0]
        u_yy = torch.autograd.grad(outputs=u_y, inputs=y, grad_outputs=torch.ones_like(u_y),
                                   retain_graph=True, create_graph=True)[0]

        res =  (u_xx + u_yy) + 5 * torch.pi**2 * torch.sin(2*torch.pi * x) * torch.sin(torch.pi * y)
        res_b = u_b - self.ub

        return res, res_b



    def loss_func(self):
        start_epoch = time.time()
        res, res_b = self.net_f(self.x, self.y, self.xb, self.yb)

        mse_loss = torch.nn.MSELoss()
        loss = mse_loss(res, torch.zeros_like(res)) + mse_loss(res_b, torch.zeros_like(res_b))

        self.optimizer.zero_grad()
        #loss.backward()
        loss.backward(retain_graph=True)


        self.iter += 1
        self.losses_hist.append(loss.item())
        print("Epoch L-BFGS [%d], Loss: %.8f, Time: %.4fs" % (self.iter, loss.item(), time.time()-start_epoch))

        return loss

    def train(self, nIter_Adam, nIter_Lbfgs):
        self.dnn.train()
        total_time = 0
        for epoch in range(nIter_Adam):
            start_epoch = time.time()

            res, res_b = self.net_f(self.x, self.y, self.xb, self.yb)

            mse_loss = torch.nn.MSELoss()
            loss = mse_loss(res, torch.zeros_like(res)) + mse_loss(res_b, torch.zeros_like(res_b))

            # Backward and optimize
            self.optimizer_Adam.zero_grad()
            #loss.backward()
            loss.backward(retain_graph=True)  # Trong vòng lặp Adam

            self.optimizer_Adam.step()

            self.losses_hist.append(loss.item())
            total_time += time.time() - start_epoch
            print("Epoch Adam [%d], Loss: %.8f, Time: %.4fs" % (epoch, loss.item(), time.time()-start_epoch))

        for epoch in range(nIter_Lbfgs):
            start_epoch = time.time()

            self.optimizer.step(self.loss_func)

        # Cập nhật tổng thời gian
            total_time += time.time() - start_epoch
        print("Total training time: %.4fs" % total_time)


    def predict(self, X):
        x = torch.tensor(X[:, 0:1]).float().to(self.device)
        y = torch.tensor(X[:, 1:2]).float().to(self.device)

        self.dnn.eval()
        u = self.net_u(x, y)
        return u


In [29]:
def exact_solution(x, y):
    u = np.sin(2 * np.pi * x) * np.sin(2 * np.pi * y)
    return u


def generate_data(x_samples, y_samples, xb_samples, yb_samples):
    data = []
    data_b = []

    for x in x_samples:
        for y in y_samples:
            if x != 0 and x != 1 and y != 0 and y != 1:
                u = exact_solution(x, y)
                data.append([x, y, u])

    for x in xb_samples:
        for y in yb_samples:
            if x == 0 or x == 1 or y == 0 or y == 1:
                data_b.append([x, y, 0])


    return np.array(data, dtype=np.float32), np.array(data_b, dtype=np.float32)

def generate_test_data(x_samples, y_samples):
    data = []

    for x in x_samples:
        for y in y_samples:
            u = exact_solution(x, y)
            data.append([x, y, u])
    return np.array(data, dtype=np.float32)

def generate_arbitrary_test_points(total_points=100, seed=42):
    np.random.seed(seed)  # Fix the seed to ensure reproducibility

    # Calculate the number of interior and boundary points based on the 3:7 ratio
    n_boundary = int(0.3 * total_points)
    n_interior = int(0.7 * total_points)

    # Generate interior points (uniform random sampling inside the domain)
    x_interior = np.random.uniform(0, 1, n_interior)
    y_interior = np.random.uniform(0, 1, n_interior)

    # Generate boundary points by sampling along the edges of the domain
    x_boundary = np.concatenate([np.zeros(n_boundary//4), np.ones(n_boundary//4), np.random.uniform(0, 1, n_boundary//4), np.random.uniform(0, 1, n_boundary//4)])
    y_boundary = np.concatenate([np.random.uniform(0, 1, n_boundary//4), np.random.uniform(0, 1, n_boundary//4), np.zeros(n_boundary//4), np.ones(n_boundary//4)])


    # Combine interior, initial and boundary points
    x_combined = np.concatenate([x_interior, x_boundary])
    y_combined = np.concatenate([y_interior, y_boundary])


    # Calculate the exact solution for each (x, y, t) pair
    u_combined = exact_solution(x_combined, y_combined)

    # Stack the (x, y, t, u) data together
    data_test = np.vstack((x_combined, y_combined, u_combined)).T

    return data_test

def data_process():
    delta_y = 0.05
    delta_x = 0.05

    delta_yb = 0.025
    delta_xb = 0.025


    # Define space and time discretization
    x_samples = np.arange(0, 1.0+delta_x, delta_x)
    y_samples = np.arange(0, 1.0+delta_y, delta_y)

    xb_samples = np.arange(0, 1.0+delta_xb, delta_xb)
    yb_samples = np.arange(0, 1.0+delta_yb, delta_yb)


    data, data_b = generate_data(x_samples, y_samples, xb_samples, yb_samples)
    return data, data_b

In [ ]:
def train():
    data, data_b = data_process()

    layers = [2, 16, 16, 16, 16, 16, 1]
    X_train = data[:,0:3]
    Xb = data_b[:, 0:2]
    ub = data_b[:,2]
    # training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PhysicsInformedNN(X_train, Xb, ub, layers, lr=1e-3, device=device)
    model.train(nIter_Adam= 1000, nIter_Lbfgs=300)

    # Test data generation
    data_test = generate_arbitrary_test_points(total_points=500, seed=42)


    # Model predictions
    u_pred = model.predict(data_test[:,0:2])

    u_test = torch.tensor(data_test[:, 2], dtype=torch.float32, requires_grad=False).to(device)
    # Calculate L2 error
    mse_loss = torch.nn.MSELoss()
    L2 = mse_loss(u_pred, u_test)

    print("L2 =", L2)

    # Plot loss history
    plot_loss(model.losses_hist, ylabel='Loss')
    return model


'''def plot_loss(losses, ylabel):
    epochs = len(losses)
    x_epochs = np.arange(1, epochs + 1)

    plt.figure()
    plt.plot(x_epochs, losses, color='blue')
    plt.xlabel('Iteration')
    plt.ylabel(ylabel)
    plt.gca().yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
    plt.savefig('loss_his.png')
    plt.show()'''
def plot_loss(losses, ylabel):
    epochs = len(losses)
    x_epochs = np.arange(1, epochs + 1)

    # Save loss history to txt file
    np.savetxt('loss_history.txt', np.column_stack((x_epochs, losses)),
               header='Epoch\tLoss', fmt=['%d', '%.10e'], delimiter='\t')

    plt.figure()
    plt.plot(x_epochs, losses, color='blue')
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)

    plt.yscale('log')  # Use log scale

    ax = plt.gca()

    # Set ticks only at powers of 10
    loc = mtick.LogLocator(base=10.0, subs=(1.0,), numticks=12)
    ax.yaxis.set_major_locator(loc)

    # Format ticks as 10^x
    formatter = mtick.FuncFormatter(lambda y, _: r'$10^{{{}}}$'.format(int(np.log10(y))))
    ax.yaxis.set_major_formatter(formatter)

    plt.grid(True, which="both", ls="--", lw=0.5)
    plt.tight_layout()
    plt.savefig('loss_his.png')
    plt.show()


if __name__ == "__main__":
    model = train()
